In [59]:
import pandas as pd
import numpy as np
import os
import glob
import geopandas as gpd

In [60]:
root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

Import the data into a standard structrue for the geopandas dataframes:

Type | SubType | Name/ID | Easting | Northing

In [205]:
def Import_Points(infile, inputs):
    print(inputs["Type"])
    #Get the before and after columns names
    usecols = list(filter(None,list(inputs.values())[1:5]))
    col_names = list({key: value for key, value in inputs.items() if value in usecols}.keys())

    #Get the data types
    t_dtypes = {key: value for key, value in dtypes.items() if key in col_names}
    dtypes_in = {inputs[k]:v for k, v in t_dtypes.items()}
    dtype_out = {key: value for key, value in dtypes.items() if key in col_names}

    #Import the data
    df = pd.read_csv(infile, usecols=usecols, encoding = "ISO-8859-1", dtype=dtypes_in) 

    #Reformat ready for next step
    df.columns = col_names
    df = df.astype(dtype_out)

    df["Type"] = inputs["Type"]

    return df

In [240]:
#source data
rail_infile = root_path + "NaPTANcsv/RailReferences.csv"
rail_inputs = {"Type":"RailwayStations", "Name":"StationName", "Easting":"Easting", "Northing":"Northing"}

ferry_infile = root_path + "NaPTANcsv/FerryReferences.csv"
ferry_inputs = {"Type":"FerryTerminals", "Name":"Name", "Easting":"Easting", "Northing":"Northing"}

bus_infile = root_path + "NaPTANcsv/Stops.csv"
bus_inputs = {"Type":"BusStops", "Name":"CommonName", "Easting":"Easting", "Northing":"Northing"}

acc2020_infile = root_path + "RoadSafety/dft-road-casualty-statistics-accident-2020.csv"
acc2020_inputs = {"Type":"RoadAccidents", "Name":"accident_reference", "Easting":"location_easting_osgr", "Northing":"location_northing_osgr"}

NSPL_infile = root_path + "NSPL_NOV_2020_UK/Data/NSPL_NOV_2020_UK.csv"
NSPL_inputs = {"Type":"NSPL", "Name":"pcd", "Easting":"oseast1m", "Northing":"osnrth1m"}

#NSUL_Infile defined below
NSUL_inputs = {"Type":"NSUL", "Name":"uprn", "Easting":"gridgb1e", "Northing":"gridgb1n"}

point_pairs = [[rail_infile, rail_inputs],
[ferry_infile, ferry_inputs],
[bus_infile, bus_inputs],
[acc2020_infile, acc2020_inputs],
[NSPL_infile,NSPL_inputs]]

NSUL_Pairs = [[root_path + "NSUL_OCT_2020/Data/" + str(i), NSUL_inputs] for i in os.listdir("D:/GeoData/NSUL_OCT_2020/Data") if ".csv" in i]


point_pairs = point_pairs + NSUL_Pairs


In [241]:
#initiate the standard structure
Points_of_Interest = pd.DataFrame({"Type":"", "Name":"", "Details_Str":"", "Details_Float":0, "Easting":0, "Northing":0}, index=[0])
dtypes = {"Type":"str", "Name":"str", "Details_Str":"str", "Details_Float":np.float64, "Easting":np.float64, "Northing":np.float64}
Points_of_Interest = Points_of_Interest.astype(dtypes)
Points_of_Interest = Points_of_Interest.drop(0)

In [242]:
for l in point_pairs:
    Points_of_Interest = Points_of_Interest.append(Import_Points(l[0], l[1]), ignore_index=True)


Points_of_Interest


RailwayStations
FerryTerminals
BusStops
RoadAccidents
NSPL
NSUL
NSUL
NSUL
NSUL
NSUL
NSUL
NSUL
NSUL
NSUL
NSUL
NSUL


,Type,Name,Details_Str,Details_Float,Easting,Northing
0,RailwayStations,Penzance Rail Station,NaN,NaN,147588.0,30599.0
1,RailwayStations,St Ives (Cornwall) Rail Station,NaN,NaN,151947.0,40127.0
2,RailwayStations,Carbis Bay Rail Station,NaN,NaN,152930.0,38745.0
3,RailwayStations,St Erth Rail Station,NaN,NaN,154150.0,35730.0
4,RailwayStations,Lelant Saltings Rail Station,NaN,NaN,154430.0,36640.0
...,...,...,...,...,...,...
42525401,NSUL,2007025533,NaN,NaN,445978.0,406341.0
42525402,NSUL,10094917555,NaN,NaN,505625.0,426924.0
42525403,NSUL,10094837365,NaN,NaN,456241.0,402766.0
42525404,NSUL,10095587191,NaN,NaN,504274.0,438612.0


In [ ]:
#Point data lefts: Land registry, fire stations, schools